In [1]:
import json
import csv

# list of users is here http://crowd.globalfishingwatch.org/api/user?limit=100

# This notebook takes the tasks json file and the tas_runs json file from 
# identify random vessels adn lines up the results, producing a csv file
# called random_vessel_classification.csv
# Make sure that the files id_random_vessels_tasks.json and id_random_vessels_task_runs.json
# are in the same directory as this file.

In [5]:
f = open('id_seismic_vessels_tasks.json', 'rU')
tasks = json.loads(f.read())
f.close()

f = open('id_seismic_vessels_task_runs.json','rU')
task_runs = json.loads(f.read())
f.close()

In [6]:
# was easier to copy and paste http://crowd.globalfishingwatch.org/api/user?limit=100 then programatically
# download it, but that also means that you probably have to go and get this data again if it is updatedçy
users =[{"locale": "en", "name": "davidkroodsma", "created": "2016-01-01T19:02:37.484806", "fullname": "David Kroodsma"}, {"locale": "en", "name": "bjornbergman", "created": "2016-01-02T17:41:56.596637", "fullname": "Bjorn Bergman"}, {"locale": "en", "name": "davidkroodsmathe2nd", "created": "2016-01-08T03:05:25.181515", "fullname": "David Kroodsma the 2nd"}, {"locale": "en", "name": "alexwilson", "created": "2016-01-08T08:38:31.791302", "fullname": "Alex Wilson"}, {"locale": "en", "name": "chris", "created": "2016-01-08T17:04:58.441540", "fullname": "Chris"}, {"locale": "en", "name": "enriquetuya", "created": "2016-01-19T15:46:18.012860", "fullname": "Enrique Tuya"}, {"locale": "en", "name": "kristinaboerder", "created": "2016-01-19T18:57:51.891476", "fullname": "Kristina Boerder"}, {"locale": "en", "name": "vaiduke2", "created": "2016-01-21T15:03:22.087286", "fullname": "Vaida Lilionyte"}, {"locale": "en", "name": "katepepler", "created": "2016-02-01T16:24:38.348003", "fullname": "Kate Pepler"}, {"locale": "en", "name": "stephanielewis", "created": "2016-02-01T16:56:45.814824", "fullname": "Stephanie Lewis"}, {"locale": "en", "name": "AlexCerra", "created": "2016-02-01T19:16:51.470060", "fullname": "Alexander Cerra"}, {"locale": "en", "name": "juliecharbonneau", "created": "2016-02-01T23:07:18.623497", "fullname": "Julie Charbonneau"}, {"locale": "en", "name": "ninagalle", "created": "2016-02-02T03:20:45.923182", "fullname": "Nina Galle"}, {"locale": "en", "name": "sidneyblack-rotchin", "created": "2016-02-02T15:15:18.889919", "fullname": "Sidney Black-Rotchin"}, {"locale": "en", "name": "daivdtest", "created": "2016-02-04T18:25:16.909258", "fullname": "David"}, {"locale": "en", "name": "cailinburmaster", "created": "2016-02-04T19:07:46.558313", "fullname": "Cailin Burmaster"}, {"locale": "en", "name": "elizabethnagel", "created": "2016-02-04T19:55:55.974887", "fullname": "Elizabeth Nagel"}, {"locale": "en", "name": "isabelfleisher", "created": "2016-02-07T21:03:22.224515", "fullname": "isabel fleisher"}, {"locale": "en", "name": "ciarawillis", "created": "2016-02-10T00:50:03.605317", "fullname": "Ciara Willis"}, {"locale": "en", "name": "clairechristie", "created": "2016-02-10T01:50:44.444883", "fullname": "Claire Christie "}]

In [14]:
results = {}
the_users = []


rows = []
for r in task_runs:
    task_id = r['task_id']
    try:
        rows.append(json.loads(r['info']))
    except:
        print r['info']
#     try:
#         vtype = json.loads(r['info'].replace("\t",""))['vesselType']
#         mmsi = json.loads(r['info'].replace("\t",""))['mmsi']
#         user = users[r['user_id']-1]['fullname']
#         if user not in the_users:
#             the_users.append(user)
#         if task_id not in results:
#             results[task_id]={}
#         results[task_id][user] = vtype
#         results[task_id]['mmsi'] = mmsi
#         results[task_id]['url'] = "http://crowd.globalfishingwatch.org/project/id_random_vessels/task/"+str(task_id)
#     except:
#         print r['info']

# print the_users




{"mmsi":"273366690","vesselType":"This vessel self-reports as a "fishing vessel" on marine traffic","search":"Yes","search_url":""}


In [15]:
import pandas as pd
df = pd.DataFrame(rows)

In [27]:
df = df.sort_values(by=['mmsi'])

In [35]:
results = {}
mmsi = 0
for index, row in df.iterrows():
    if row['mmsi'] != mmsi:
        mmsi = row['mmsi']
        results[mmsi] = []
    results[mmsi].append(row['vesselType'])
    

In [39]:
with open("results.txt", 'w') as f:
    for mmsi in results:
        f.write(mmsi)
        if len(set(results[mmsi])) == 1:
            f.write("\t"+results[mmsi][0])
        else:
            f.write("\tdisagree")
        for t in results[mmsi]:
            f.write("\t"+t)
        f.write("\n")


In [5]:
for r in results:
    for t in the_users:
        if t not in results[r]:
            results[r][t]=''

In [6]:
header = ['url','mmsi'] + the_users
rows = []
for r in results:
    row = [results[r]['url'],results[r]['mmsi']]
    for t in the_users:
        row += [results[r][t]]
    rows.append(row)



In [7]:
with open('random_vessel_classification.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)
